In [14]:
from google.colab import files
uploaded = files.upload()

In [2]:
import numpy as np

In [210]:
hsearch = np.genfromtxt('hsearch.csv',delimiter=',')
als19 = np.genfromtxt('als_submission_19.csv',delimiter=',')
dct = np.genfromtxt('decision_tree_classifier_submission.csv',delimiter=',')
gbt = np.genfromtxt('gradient_boosted_tree_classifier_submission.csv',delimiter=',')
logreg = np.genfromtxt('logistic_regression_submission.csv',delimiter=',')
rfc = np.genfromtxt('random_forest_classifier_submission.csv',delimiter=',')

in1 = [hsearch,als19,dct,gbt,logreg,rfc]

in1[0][0][0] = 199810105760 #Band-Aid™

in1 = np.asarray(in1)

#S = np.asarray(S)

In [39]:
def clean_input(nparray):
  #nparray = np.delete(nparray,0,0) #delete header row
  nparray[:,1] = 2*nparray[:,1]-1  #map {0,1} -> {-1,1} (using function 2n-1)
  #nparray = nparray[np.argsort(nparray[:,0])] #sort by userid_trackid and return
  #nparray = np.delete(nparray,0,1) #delete userid_trackid column
  return nparray

In [243]:
def get_ratings(nparray):
  nparray = np.transpose(nparray)
  nparray = np.delete(nparray,0,0)
  #nparray = np.transpose(nparray)
  nparray = nparray*2 - 1
  return nparray

In [286]:
S = np.empty([120000,6])
print(S)
print(S.shape)

[[ 1.  1.  1. -1. -1. -1.]
 [ 1.  1. -1.  1. -1. -1.]
 [ 1. -1. -1. -1.  1.  1.]
 ...
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
(120000, 6)


In [287]:
S[:,0] = get_ratings(hsearch)
S[:,1] = get_ratings(als19)
S[:,2] = get_ratings(dct)
S[:,3] = get_ratings(gbt)
S[:,4] = get_ratings(logreg)
S[:,5]= get_ratings(rfc)

In [288]:
S

array([[ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1., -1., -1., -1., -1.],
       ...,
       [-1.,  1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.]])

In [289]:
S.shape

(120000, 6)

In [293]:
St = np.transpose(S)
St

array([[ 1.,  1.,  1., ..., -1., -1., -1.],
       [ 1.,  1.,  1., ...,  1., -1., -1.],
       [ 1.,  1., -1., ..., -1., -1., -1.],
       [ 1.,  1., -1., ..., -1., -1., -1.],
       [ 1.,  1., -1., ..., -1., -1., -1.],
       [ 1.,  1., -1., ..., -1., -1., -1.]])

In [294]:
St.shape

(6, 120000)

In [310]:
#StS = np.multiply(St,S)
#St * S
StS = np.dot(St,S)

print(StS)
print(StS.shape)


[[120000.  42332.  66472.  72296.  73072.  72436.]
 [ 42332. 120000.  41552.  42876.  42980.  42884.]
 [ 66472.  41552. 120000. 112444. 112328. 112856.]
 [ 72296.  42876. 112444. 120000. 119164. 119196.]
 [ 73072.  42980. 112328. 119164. 120000. 119300.]
 [ 72436.  42884. 112856. 119196. 119300. 120000.]]
(6, 6)


In [312]:
StSinv = np.linalg.inv(StS)

print(StSinv)
print(StSinv.shape)

[[ 1.37582753e-05 -2.15198041e-06  1.89371885e-06  1.46188039e-06
  -1.07709081e-05 -6.08949302e-08]
 [-2.15198041e-06  9.90777000e-06 -1.10385283e-06 -9.05355438e-07
  -2.91767662e-07 -1.42075934e-08]
 [ 1.89371885e-06 -1.10385283e-06  7.28521415e-05 -1.56390075e-05
  -5.32656534e-07 -5.31998668e-05]
 [ 1.46188039e-06 -9.05355438e-07 -1.56390075e-05  7.80433789e-04
  -3.70269542e-04 -3.92946176e-04]
 [-1.07709081e-05 -2.91767662e-07 -5.32656534e-07 -3.70269542e-04
   9.00609191e-04 -5.20460008e-04]
 [-6.08949302e-08 -1.42075934e-08 -5.31998668e-05 -3.92946176e-04
  -5.20460008e-04  9.66145298e-04]]
(6, 6)


In [321]:
STx = np.empty([6,1])
STx
STx[0] = 120000*((2*0.84150)-1) #hsearch
STx[1] = 120000*((2*0.72655)-1) #als
STx[2] = 120000*((2*0.82763)-1) #dct
STx[3] = 120000*((2*0.84830)-1) #gbt
STx[4] = 120000*((2*0.84908)-1) #logreg
STx[5] = 120000*((2*0.84864)-1) #rfc

STx


array([[81960. ],
       [54372. ],
       [78631.2],
       [83592. ],
       [83779.2],
       [83673.6]])

In [323]:
aLS = np.dot(StSinv,STx)
aLS

array([[0.37425429],
       [0.17421835],
       [0.02029599],
       [0.17878985],
       [0.01145238],
       [0.20104256]])

In [328]:
Sensemble = np.dot(S,aLS)
print(Sensemble)
print(Sensemble.shape)

[[ 0.96005343]
 [ 0.96005343]
 [ 0.13689185]
 ...
 [-0.61161673]
 [-0.96005343]
 [-0.96005343]]
(120000, 1)


In [331]:
np.savetxt("foo.csv",Sensemble,delimiter=',')